In [1]:
import cc3d
import numpy as np
# from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
# import matplotlib.pyplot as plt
from plotly import graph_objs as go
import nibabel as nib
from dataloader import extract_spleen_coord

In [2]:
file_id = '0061'

In [3]:
path_label= r'D:\Data\cs-8395-dl\output\assignment3\segmentation_prob\label{}.nii.gz'.format(file_id)
label_nib = nib.load(path_label)
label_3d =label_nib.get_fdata()

In [4]:
label_bin = label_3d>label_3d.max()/2
label_bin = label_bin.astype(np.uint16)

In [5]:
# np.random.seed(42)
labels_out = cc3d.connected_components(label_bin, 
                                       connectivity=6,
                                       out_dtype=np.uint16)

In [6]:
labels_out.shape, np.unique(labels_out.flatten())

((512, 512, 138),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
       dtype=uint16))

In [7]:
path_axial_prob = r'D:\Data\cs-8395-dl\output\assignment3\axial_spleen_prob\{}.npy'.format(file_id)
axial_prob = np.load(path_axial_prob)

In [8]:
axial_prob.shape

(138, 2)

In [14]:
spleen_prob

0.014490343

In [16]:
# You can extract individual components like so:
N = np.max(labels_out)
criteria = 0
for segid in range(1, N+1):
    extracted_image = labels_out * (labels_out == segid)
    vol = (extracted_image!=0).sum()/(extracted_image.size)
    _,_,z=extract_spleen_coord(extracted_image,label=segid)
    spleen_prob=np.median(axial_prob[z[0]:z[1],1])
    metric = vol*spleen_prob
    print('segid: {}, Volume ratio: {:.6f}, median prob {:.2f}, vol*prob {:.6f}'.format(segid,vol, spleen_prob,metric))
    if metric>criteria:
        print('updating criteria {:.6f}-->{:.6f} for seg_id {}'.format(criteria,metric, segid))
        criteria=metric
    

segid: 1, Volume ratio: 0.000035, median prob 0.01, vol*prob 0.000001
updating criteria 0.000000-->0.000001 for seg_id 1
segid: 2, Volume ratio: 0.000377, median prob 0.07, vol*prob 0.000025
updating criteria 0.000001-->0.000025 for seg_id 2
segid: 3, Volume ratio: 0.000087, median prob 0.04, vol*prob 0.000003
segid: 4, Volume ratio: 0.005346, median prob 0.03, vol*prob 0.000176
updating criteria 0.000025-->0.000176 for seg_id 4
segid: 5, Volume ratio: 0.000004, median prob 0.02, vol*prob 0.000000
segid: 6, Volume ratio: 0.000004, median prob 0.03, vol*prob 0.000000
segid: 7, Volume ratio: 0.000001, median prob 0.03, vol*prob 0.000000
segid: 8, Volume ratio: 0.000000, median prob 0.04, vol*prob 0.000000
segid: 9, Volume ratio: 0.000043, median prob 0.03, vol*prob 0.000001
segid: 10, Volume ratio: 0.000021, median prob 0.03, vol*prob 0.000001
segid: 11, Volume ratio: 0.000005, median prob 0.03, vol*prob 0.000000
segid: 12, Volume ratio: 0.000079, median prob 0.04, vol*prob 0.000003
segi

In [17]:
segid

32

In [42]:
spleen_comp = labels_out*(labels_out == 2)+labels_out*(labels_out == 4)+labels_out*(labels_out == 29)

In [43]:
spleen_comp.max(),spleen_comp.min()

(29, 0)

In [44]:
path_save = r'D:\Data\cs-8395-dl\output\assignment3\segmentation_cc_thresh\label{}.nii.gz'.format(file_id)

In [45]:
label_spleen = nib.Nifti1Image(spleen_comp,label_nib.affine,label_nib.header)
nib.save(label_spleen,path_save)